In [20]:
!pip install ffmpeg moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import math
import moviepy.editor as mp
from operator import itemgetter

In [22]:
my_clip = mp.VideoFileClip('drive/MyDrive/dataset/2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4')

audio = my_clip.audio
# my_clip.audio.write_audiofile('drive/MyDrive/dataset/audio/2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp3')


In [28]:
def find_speaking(audio_clip, window_size=20, volume_threshold=0.0001):

    num_windows = math.floor(audio_clip.end/window_size)

    window_is_silent = []
    for i in range(num_windows):
        s = audio_clip.subclip(i * window_size, (i + 1) * window_size)

        v = s.max_volume()
        window_is_silent.append(v < volume_threshold)

    start = 0
    end = 0
    speaking_intervals = []
    for i in range(1, len(window_is_silent)):
        if window_is_silent[i - 1] and not window_is_silent[i]:
            start = i * window_size

        if not window_is_silent[i - 1] and window_is_silent[i]:
            end = i * window_size
            new_speaking_interval = [start - 0.25, end + 0.25]

            need_to_merge = len(speaking_intervals) > 0 and speaking_intervals[-1][1] > new_speaking_interval[0]
            if need_to_merge:
                merged_interval = [speaking_intervals[-1][0], new_speaking_interval[1]]
                speaking_intervals[-1] = merged_interval
            else:
                element = (new_speaking_interval, new_speaking_interval[1] - new_speaking_interval[0])
                speaking_intervals.append(element)

    return speaking_intervals

In [32]:
def find_longest_speaking_interval(speaking_intervals):    
    maximum = max(speaking_intervals, key=itemgetter(1))[0] 
    return maximum 

In [34]:
result = find_longest_speaking_interval(find_speaking(audio))
print(result[0])
print(result[1])

839.75
5580.25
